# Collect Data

- [Sentiment140](https://www.kaggle.com/datasets/krishbaisoya/tweets-sentiment-analysis) contains 1,600,000 tweets extracted using the twitter API . The tweets have been annotated (0 = negative, 1 = positive) and they can be used to detect sentiment.

In [1]:
import os
import sys

import pandas as pd
from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# print(notebook_dir)
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Load Data

In [3]:
base_path = os.path.join(notebook_dir, '../data/', 'twitter/')
full_path = os.path.join(base_path, 'train_data.csv')

df = DataProcessing.load_from_file(full_path, 'csv')
df

,sentence,sentiment
0,awww that s a bummer you shoulda got david carr of third day to do it d,0
1,is upset that he can t update his facebook by texting it and might cry as a result school today also blah,0
2,i dived many times for the ball managed to save the rest go out of bounds,0
3,my whole body feels itchy and like its on fire,0
4,no it s not behaving at all i m mad why am i here because i can t see you all over there,0
...,...,...
1523970,just woke up having no school is the best feeling ever,1
1523971,thewdb com very cool to hear old walt interviews,1
1523972,are you ready for your mojo makeover ask me for details,1
1523973,happy th birthday to my boo of alll time tupac amaru shakur,1


In [19]:
filt_sports = (df.Sentence == 'nba')
df[filt_sports]

,Sentence,Label,Model


In [4]:
sentences = DataProcessing.df_to_list(df, 'sentence')
sentences[: 3]

['awww that s a bummer you shoulda got david carr of third day to do it d',
 'is upset that he can t update his facebook by texting it and might cry as a result school today also blah',
 'i dived many times for the ball managed to save the rest go out of bounds']

In [5]:
prediction_properties = """A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:

    1. <p_s>, any source entity in the domain.
        - Can be a person (with a name) or a  person such as a  reporter,  analyst,  expert,  top executive,  senior level person, etc), civilian.
        - Can only be an organization that is associated with the  prediction.
    2. <p_t>, any target entity in the  domain.
	    - Can be a person (with a name) or a  person such as a  reporter,  analyst,  expert,  top executive,  senior level person, etc).
        - Can only be an organization that is associated with the  prediction.
    3. <p_d>, date or time range when <p> is expected to come to fruition or when one should observe the <p>.
        - Forecast can range from a second to anytime in the future.
        - Answers the questions: "How far to go out from today?" or "Where to stop?".
    4. <p_o>,  prediction outcome.
        - Details relevant details such as outcome, a quantifiable metric, or slope.
"""

In [6]:
tgmf = TextGenerationModelFactory()

# Groq Cloud (https://console.groq.com/docs/overview)
gemma_29b_generation_model = tgmf.create_instance('gemma2-9b-it') 
llama_318b_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant') 
llama_3370b_versatile_generation_model = tgmf.create_instance('llama-3.3-70b-versatile')  
llama_guard_4_12b_generation_model = tgmf.create_instance('meta-llama/llama-guard-4-12b')  

models = [gemma_29b_generation_model, llama_318b_instant_generation_model, llama_3370b_versatile_generation_model, llama_guard_4_12b_generation_model]

In [7]:
def llm_as_a_judge(sentences: list, prediction_properties: str):
    
    labels = []
    for sentences_idx in tqdm(range(len(sentences))):
        sentence = sentences[sentences_idx]

        prompt = f"Classify the sentence: '{sentence}' as a prediction or non-prediction. I define a prediction with the following properties: {prediction_properties}. Only state a binary label of prediction or non-prediction and nothing else. Do NOT use the word 'safe' and keep label lower case." 
        if sentences_idx < 3:
            print(f"Prompt: {prompt}")
            # print()

        for model in models:  
            input_prompt = model.user(prompt)
            raw_text_llm_generation = model.chat_completion([input_prompt])
    
            for label in raw_text_llm_generation.split("\n"):
                if label.strip():
                    sentence_to_label = (sentence, label, model.__name__())
            labels.append(sentence_to_label)
    return labels

In [13]:
sentence_label = llm_as_a_judge(sentences[33:100], prediction_properties)

  0%|          | 0/67 [00:00<?, ?it/s]

Prompt: Classify the sentence: 'awe i love you too am here i miss you' as a prediction or non-prediction. I define a prediction with the following properties: A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:

    1. <p_s>, any source entity in the domain.
        - Can be a person (with a name) or a  person such as a  reporter,  analyst,  expert,  top executive,  senior level person, etc), civilian.
        - Can only be an organization that is associated with the  prediction.
    2. <p_t>, any target entity in the  domain.
	    - Can be a person (with a name) or a  person such as a  reporter,  analyst,  expert,  top executive,  senior level person, etc).
        - Can only be an organization that is associated with the  prediction.
    3. <p_d>, date or time range when <p> is expected to come to fruition or when one should observe the <p>.
        - Forecast can range from a second to anytime in the future.
        - Answers the ques

  1%|▏         | 1/67 [00:02<02:22,  2.16s/it]

Prompt: Classify the sentence: 'i cry my asian eyes to sleep at night' as a prediction or non-prediction. I define a prediction with the following properties: A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:

    1. <p_s>, any source entity in the domain.
        - Can be a person (with a name) or a  person such as a  reporter,  analyst,  expert,  top executive,  senior level person, etc), civilian.
        - Can only be an organization that is associated with the  prediction.
    2. <p_t>, any target entity in the  domain.
	    - Can be a person (with a name) or a  person such as a  reporter,  analyst,  expert,  top executive,  senior level person, etc).
        - Can only be an organization that is associated with the  prediction.
    3. <p_d>, date or time range when <p> is expected to come to fruition or when one should observe the <p>.
        - Forecast can range from a second to anytime in the future.
        - Answers the ques

  3%|▎         | 2/67 [00:03<01:50,  1.70s/it]

Prompt: Classify the sentence: 'ok i m sick and spent an hour sitting in the shower cause i was too sick to stand and held back the puke like a champ bed now' as a prediction or non-prediction. I define a prediction with the following properties: A prediction <p> = (<p_s>, <p_t>, <p_d>, <p_o>), where it consists of the following four properties:

    1. <p_s>, any source entity in the domain.
        - Can be a person (with a name) or a  person such as a  reporter,  analyst,  expert,  top executive,  senior level person, etc), civilian.
        - Can only be an organization that is associated with the  prediction.
    2. <p_t>, any target entity in the  domain.
	    - Can be a person (with a name) or a  person such as a  reporter,  analyst,  expert,  top executive,  senior level person, etc).
        - Can only be an organization that is associated with the  prediction.
    3. <p_d>, date or time range when <p> is expected to come to fruition or when one should observe the <p>.
       

100%|██████████| 67/67 [03:17<00:00,  2.94s/it]


In [9]:
sentence_label

[('awww that s a bummer you shoulda got david carr of third day to do it d',
  'non-prediction ',
  'gemma2-9b-it'),
 ('awww that s a bummer you shoulda got david carr of third day to do it d',
  'non-prediction',
  'llama-3.1-8b-instant'),
 ('awww that s a bummer you shoulda got david carr of third day to do it d',
  'non-prediction',
  'llama-3.3-70b-versatile'),
 ('awww that s a bummer you shoulda got david carr of third day to do it d',
  'S13',
  'meta-llama/llama-guard-4-12b'),
 ('is upset that he can t update his facebook by texting it and might cry as a result school today also blah',
  'non-prediction ',
  'gemma2-9b-it'),
 ('is upset that he can t update his facebook by texting it and might cry as a result school today also blah',
  'non-prediction',
  'llama-3.1-8b-instant'),
 ('is upset that he can t update his facebook by texting it and might cry as a result school today also blah',
  'non-prediction',
  'llama-3.3-70b-versatile'),
 ('is upset that he can t update his face

In [14]:
sentence_label_df = pd.DataFrame(sentence_label, columns=['Sentence', 'Label', 'Model'])
sentence_label_df


,Sentence,Label,Model
0,awe i love you too am here i miss you,non-prediction,gemma2-9b-it
1,awe i love you too am here i miss you,non-prediction,llama-3.1-8b-instant
2,awe i love you too am here i miss you,non-prediction,llama-3.3-70b-versatile
3,awe i love you too am here i miss you,safe,meta-llama/llama-guard-4-12b
4,i cry my asian eyes to sleep at night,non-prediction,gemma2-9b-it
...,...,...,...
263,you don t follow me either and i work for you,safe,meta-llama/llama-guard-4-12b
264,a bad nite for the favorite teams astros and spartans lose the nite out with t w was good,non-prediction,gemma2-9b-it
265,a bad nite for the favorite teams astros and spartans lose the nite out with t w was good,non-prediction,llama-3.1-8b-instant
266,a bad nite for the favorite teams astros and spartans lose the nite out with t w was good,non-prediction,llama-3.3-70b-versatile


In [15]:
filt_prediction = (sentence_label_df.Label == 'prediction')
sentence_label_df[filt_prediction]

,Sentence,Label,Model
98,i have a sad feeling that dallas is not going to show up i gotta say though you d think more shows would use music from the game mmm,prediction,llama-3.3-70b-versatile
126,i had such a nice day too bad the rain comes in tomorrow at am,prediction,llama-3.3-70b-versatile
134,damm back to school tomorrow,prediction,llama-3.3-70b-versatile
